In [1]:
!nvidia-smi

Wed May 14 16:38:11 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:03:00.0 Off |                    0 |
| N/A   31C    P0              69W / 700W |     13MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from models.transformers import DecoderOnlyTransformer
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import yaml
import gc
import torch
from torchinfo import summary
import os
import json
import shutil
import wandb
import yaml
from data.datasets import CFGDataset, verify_dataloader
from trainers.GPT_trainer import GPT_Trainer
from trainers.utils import create_optimizer, create_scheduler
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

#Read config yaml file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
print(f"Config loaded successfully.")


import os
os.system("export WANDB_DIR=\tmp")
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


ModuleNotFoundError: No module named 'models'

#### Loading Data

In [2]:
train_dataset =  CFGDataset(
    data_file="cfg_sentences_train_cfg3b.npy", 
    batch_size = config["data"]["batch_size"],
    seq_len = config["data"]["seq_len"],
    eos_token = config["data"]["eos_token"],
    sos_token = config["data"]["sos_token"],
    ) 

val_dataset =  CFGDataset(
    data_file="cfg_sentences_val_cfg3b.npy", 
    
    batch_size = config["data"]["batch_size"],
    seq_len = config["data"]["seq_len"],
    eos_token = config["data"]["eos_token"],
    sos_token = config["data"]["sos_token"],
    ) 


train_loader = DataLoader(train_dataset, 
                          batch_size = None, 
                          num_workers=config["data"]["NUM_WORKERS"] if device == "cuda" else 0, 
                          pin_memory=True)

val_loader = DataLoader(val_dataset, 
                        batch_size=None, 
                        num_workers=config["data"]["NUM_WORKERS"] if device == "cuda" else 0,
                        pin_memory=True)

verify_dataloader(train_loader)
print("_"*50)
verify_dataloader(val_loader)
#print("="*50)
#print("Verify Validation DataLoader")
#verify_dataloader(val_loader)

Verifying dataloader...
Number of batches:  40669
Total number of tokens:  1.998977e+09
Example batch shapes (shifted, golden):  torch.Size([96, 512]) torch.Size([96, 512])
__________________________________________________
Verifying dataloader...
Number of batches:  4518
Total number of tokens:  2.221086e+08
Example batch shapes (shifted, golden):  torch.Size([96, 512]) torch.Size([96, 512])


In [4]:
model_config = {}
model_config = config['model'].copy()

model_config.update({
    'num_classes': config["data"]["vocab_size"] +2 ,#include SOS and ESO tolen
    'seq_len': config["data"]["seq_len"],
    })

model = DecoderOnlyTransformer(**model_config)

for batch in train_loader:
    shifted_transcripts, golden_transcripts = batch
    print("Shape of shifted_transcripts : ", shifted_transcripts.shape)
    print("Shape of golden_transcripts  : ", golden_transcripts.shape)
    break

#model_stats = summary(model, input_data=[shifted_transcripts])
#print(model_stats)
print(f"Total parameters in the model: {sum(p.numel() for p in model.parameters()):,}")
#torch._dynamo.config.skip_nnmodule_hook_guards = False
#model = torch.compile(model, mode="default", fullgraph=True)


Shape of shifted_transcripts :  torch.Size([96, 512])
Shape of golden_transcripts  :  torch.Size([96, 512])
Total parameters in the model: 85,059,845


In [5]:
from trainers.utils.create_scheduler import plot_lr_schedule


torch.cuda.empty_cache()
gc.collect()
#wandb.finish()
trainer = GPT_Trainer(
    model = model, 
    config = config,
    config_file = "config.yaml",
    run_name = "full_rope_86", 
    device = device
)
wandb.watch(trainer.model, log="all")

trainer.optimizer = create_optimizer.create_optimizer(
    model = model, 
    opt_config = config["optimizer"]
)

trainer.scheduler = create_scheduler.create_scheduler(
    optimizer=trainer.optimizer,
    scheduler_config=config['scheduler'],
    train_loader=train_loader,
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps']
)

#plot_lr_schedule(trainer.scheduler, num_epochs=len(train_loader), train_loader=train_loader)

#trainer.load_checkpoint("/ocean/projects/cis250019p/sfragara/lstm/expts/test/checkpoints/checkpoint-best-metric-model.pth")

Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lucasfragara (teamlsfr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



🔧 Configuring Optimizer:
├── Type: ADAMW
├── Base LR: 0.0003
├── Weight Decay: 0.1
├── Parameter Groups:
│   ├── Group: self_attn
│   │   ├── LR: 0.0003
│   │   └── Patterns: []
│   ├── Group: ffn
│   │   ├── LR: 0.0003
│   │   └── Patterns: []
│   └── Default Group (unmatched parameters)
└── AdamW Specific:
    ├── Betas: [0.9, 0.98]
    ├── Epsilon: 1e-08
    └── AMSGrad: False


/jet/home/sfragara/.conda/envs/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



📈 Configuring Learning Rate Scheduler:
├── Type: COSINE
├── Cosine Annealing Settings:
│   ├── T_max: 40669 steps
│   └── Min LR: 1e-08
└── Warmup: Disabled


In [ ]:
trainer.model.train()

#trainer._validate_epoch(val_loader)
trainer.train_epoch(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
)

generating
Using default generation config...
Generating with greedy search...


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


wandb metrics being logged:  {'train/ce_loss_token': 1.1724136917890904, 'train/perplexity_token': 3.229779005050659, 'val/ce_loss_token': 1.0457908771932125, 'val/perplexity_token': 2.8456482887268066, 'learning_rate': 0.000299887684217008}

📊 Metrics (Epoch 500):
├── TRAIN:
│   ├── ce_loss_token: 1.1724
│   └── perplexity_token: 3.2298
└── VAL:
    ├── ce_loss_token: 1.0458
    └── perplexity_token: 2.8456
└── TRAINING:
    └── learning_rate: 0.000300


[Training LM]:   1%|▋                                                      | 527/40669 [02:39<2:58:28,  3.75it/s, acc_step=1/1, ce_loss_token=1.1665, lr=0.000300, perplexity_token=3.2107]

generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.9388399131295205, 'train/perplexity_token': 2.5570132732391357, 'val/ce_loss_token': 0.7947334852069616, 'val/perplexity_token': 2.213850736618042, 'learning_rate': 0.00029720941233809124}

📊 Metrics (Epoch 2500):
├── TRAIN:
│   ├── ce_loss_token: 0.9388
│   └── perplexity_token: 2.5570
└── VAL:
    ├── ce_loss_token: 0.7947
    └── perplexity_token: 2.2139
└── TRAINING:
    └── learning_rate: 0.000297


...

generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.7604975875346304, 'train/perplexity_token': 2.139340400695801, 'val/ce_loss_token': 0.7443153038620949, 'val/perplexity_token': 2.1049997806549072, 'learning_rate': 9.656511485433034e-06}

📊 Metrics (Epoch 36000):
├── TRAIN:
│   ├── ce_loss_token: 0.7605
│   └── perplexity_token: 2.1393
└── VAL:
    ├── ce_loss_token: 0.7443
    └── perplexity_token: 2.1050
└── TRAINING:
    └── learning_rate: 0.000010


generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.7602735502383947, 'train/perplexity_token': 2.1388611793518066, 'val/ce_loss_token': 0.7442119158804417, 'val/perplexity_token': 2.1047821044921875, 'learning_rate': 7.717614616055162e-06}

📊 Metrics (Epoch 36500):
├── TRAIN:
│   ├── ce_loss_token: 0.7603
│   └── perplexity_token: 2.1389
└── VAL:
    ├── ce_loss_token: 0.7442
    └── perplexity_token: 2.1048
└── TRAINING:
    └── learning_rate: 0.000008


generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.7600555353988032, 'train/perplexity_token': 2.138395071029663, 'val/ce_loss_token': 0.7441726382821798, 'val/perplexity_token': 2.1046993732452393, 'learning_rate': 5.990956738771035e-06}

📊 Metrics (Epoch 37000):
├── TRAIN:
│   ├── ce_loss_token: 0.7601
│   └── perplexity_token: 2.1384
└── VAL:
    ├── ce_loss_token: 0.7442
    └── perplexity_token: 2.1047
└── TRAINING:
    └── learning_rate: 0.000006


generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.7598410588271306, 'train/perplexity_token': 2.1379363536834717, 'val/ce_loss_token': 0.7441199235618114, 'val/perplexity_token': 2.104588508605957, 'learning_rate': 4.47911337442937e-06}

📊 Metrics (Epoch 37500):
├── TRAIN:
│   ├── ce_loss_token: 0.7598
│   └── perplexity_token: 2.1379
└── VAL:
    ├── ce_loss_token: 0.7441
    └── perplexity_token: 2.1046
└── TRAINING:
    └── learning_rate: 0.000004


[Training LM]:  93%|█████████████████████████████████████████████████▎   | 37861/40669 [3:26:02<12:23,  3.78it/s, acc_step=1/1, ce_loss_token=0.7597, lr=0.000004, perplexity_token=2.1376]